
# Práctica 1 - Distancia léxica. Aplicación a la corrección automática.

En esta práctica se pretende mostrar algunas librerías para la aplicación de medidas de distancia léxica como Levenshtein y cómo puede aplicarse a la corrección automática de errores "non real words" o corrección ortográfica.



In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## 1- Calculando distancias de Levenshtein
En este apartado vamos a calcular las distancias de Levenshtein de varias palabras y hacer una simple corrección ortográfica.

In [2]:
!pip install python-Levenshtein


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.6 MB/s eta 0:00:00


## Cálculo distancia de edición para obtener la similitud entre dos cadenas de carácteres:

In [3]:
import Levenshtein

# Ejemplo de uso de la distancia de Levenshtein
string1 = "burro"
string2 = "vurro"

distance = Levenshtein.distance(string1, string2)

print(f"La distancia de Levenshtein entre '{string1}' y '{string2}' es: {distance}")

# Probamos otras palabras


La distancia de Levenshtein entre 'burro' y 'vurro' es: 1


In [4]:
#Podemos calcular la distancia normalizada

def normalized_levenshtein(str1, str2):
  distance = Levenshtein.distance(str1, str2)
  max_len = max(len(str1), len(str2))
  if max_len == 0:
    return 0
  return distance / max_len

# Ejemplo de uso
string1 = "burro"
string2 = "vurro"

normalized_distance = normalized_levenshtein(string1, string2)
print(f"La distancia de Levenshtein normalizadas entre  '{string1}' and '{string2}' is: {normalized_distance}")

# Otro example
string3 = "kitten"
string4 = "sitting"
normalized_distance = normalized_levenshtein(string3, string4)
print(f"La distancia de Levenshtein normalizadas entre '{string3}' and '{string4}' is: {normalized_distance}")

La distancia de Levenshtein normalizadas entre  'burro' and 'vurro' is: 0.2
La distancia de Levenshtein normalizadas entre 'kitten' and 'sitting' is: 0.42857142857142855


## Podemos obtener la distancia absoluta(nº operaciones de transformación solamente) y la distancia normalizada(nº operaciones de transformación teniendo en cuenta la longitud de la secuencia de carácteres más grande entre la source-target)

In [5]:
# Ejemplo de aplicación a la corrección automática (simple)
palabras_correctas = ["hola", "mundo", "python", "pitón"]
palabra_introducida = "pyton"

mejor_candidato = ""
min_distancia = float('inf')

for palabra in palabras_correctas:
  distancia = Levenshtein.distance(palabra_introducida, palabra)
  if distancia < min_distancia:
    min_distancia = distancia
    mejor_candidato = palabra

print(f"Palabra introducida: {palabra_introducida}")
print(f"Posible corrección: {mejor_candidato}")

Palabra introducida: pyton
Posible corrección: python


## 2. Corrección ortográfica usando algunas librerías
En este apartado vamos a ver distintas librerías para la corrección ortográfica que usan distancias léxicas.

### Creación de los datos a corregir.

Se proporcionan dos textos en dos versiones, una sin errores ootográficos y otra en la que se han introducido errores que vamos a intentar corregir mediante el uso de las librerías symspellpy y hunspell. En el segundo texto, algunos de los errores introducidos se corresponden con "real words".

Los textos se proporcionan en un dataframe de pandas con objeto de ir añadiendo los resultados obtenidos y poder razonar sobre ellos más adelante.

Se proporcionan también dos ejemplos de almacenamiento y recupoaración de estos datos en y desde un fichero csv.

In [6]:
# Creación de los datos y almacenamiento en fichero:

import pandas as pd
import csv

data = pd.DataFrame(
 {
    "textos_sin_errores":
    [
    " La toma de posesión de Donald Trump"
    " como presidente de Estados Unidos se llevará"
    " a cabo el lunes veinte de enero en el Capitolio"
    " de Washington D.C. Esta ceremonia histórica,"
    " que se realiza cada cuatro años, tiene en"
    " su lista de invitados a figuras clave en la"
    " política mundial, además de empresarios que"
    " buscan ganarse el favor del nuevo comandante"
    " en jefe, y celebridades que lo han apoyado y"
    " que comparten su visión para el futuro de la"
    " unión americana."
    ,

    " Cerrar podrá mis ojos la postrera"
    " sombra, que me llevare el blanco día,"
    " y podrá desatar esta alma mía"
    " hora, a su afán ansioso linsojera;"
    " mas no de esa otra parte en la ribera"
    " dejará la memoria en donde ardía;"
    " nadar sabe mi llama la agua fría,"
    " y perder el respeto a ley severa;"
    " Alma a quien todo un Dios prisión ha sido,"
    " venas que humor a tanto fuego han dado,"
    " médulas que han gloriosamente ardido,"
    " su cuerpo dejarán, no su cuidado;"
    " serán ceniza, mas tendrán sentido."
    " Polvo serán, mas polvo enamorado."
    ],

  "textos_con_errores" :
    [
    " La tma de posesión de Donald Trump"                # toma         -> tma
    " como presdente de Estados Unidos se llevará"       # presidente   -> presdente
    " a cabo el lunes, veinte de enero en el Capittolio" # Captitolio   -> Capittolio
    " de Washington D.C. Esta ceremonia hitórrica,"      # histórica    -> hitórrica
    " que se realiza cada cuatro años, teine en"         # tiene        -> teine
    " su pista de invitados a figuras clave en la"       # lista        -> pista
    " política mundial, ademas de empresarios que"       # además       -> ademas
    " busca ganarse el fvaor del nuevo comandante"       # favor        -> fvaor
    " en jefe, y selevridades que lo han apoyado y"      # celebridades -> selevridades
    " que comparten su vission para el futuro de la"     # visión       -> vission
    " unión americana.",

    " Cerrar podrá mis hojos la postrera"          # ojos         -> hojos
    " sombra, que me llevre el blanco día,"        # llevare      -> llevre
    " y podrá desatar esta alma mía"
    " hora, a su afán ansioso lisonjera;"
    " mas no de esa otra parte en la rivrera"      # rivera        -> rivrera
    " dejará la memorria en donde ardía;"          # memoria       -> memorria
    " nadar sabe mi llama la agua fría,"
    " y preder el respeto a ley severa;"           # perder        -> preder
    " Alma a quien todo un Dios prisión ha sido,"
    " venas que rumor a tanto fuego han dado,"     # humor         -> rumor
    " médulas que han glorsamente ardido,"         # gloriosamente -> glorsamente
    " su cuerpo dejaran, no su cuidado;"           # dejarán       -> dejaran
    " serán cenisa, mas tendrán sentido."          # ceniza        -> cenisa
    " Polvo serán, mas polbu enamorado."           # polvo         -> polbu
    ]
 }
)




Si quiere guardar el dataframe en un csv puede seguir este ejemplo definiendo el path correspondiente a su sistema.

In [7]:
# Almacenamiento de datos en fichero csv
data_dir_path = ""
data_file_path = data_dir_path + "textos_p1_01.csv"
data.to_csv(data_file_path, index=False, quoting=csv.QUOTE_ALL)
data.head()


,textos_sin_errores,textos_con_errores
0,La toma de posesión de Donald Trump como pres...,La tma de posesión de Donald Trump como presd...
1,"Cerrar podrá mis ojos la postrera sombra, que...","Cerrar podrá mis hojos la postrera sombra, qu..."


Para reconstruir el dataframe a partir de un csv puede seguir este ejemplo.

In [8]:
import pandas as pd
import csv

data_file_path =  data_dir_path + "textos_p1_01.csv"
datos = pd.read_csv(data_file_path, encoding="UTF-8")
datos.head()

,textos_sin_errores,textos_con_errores
0,La toma de posesión de Donald Trump como pres...,La tma de posesión de Donald Trump como presd...
1,"Cerrar podrá mis ojos la postrera sombra, que...","Cerrar podrá mis hojos la postrera sombra, qu..."


2.- Instalación de symspellpy.

El primero de los correctores ortográficos que vamos a utilizar es symspellpy (https://symspellpy.readthedocs.io/en/latest/index.html#)

symspellpy tiene una API relativamente simple y según sus creadores es muy eficiente tanto en términos de velocidad como de uso de memoria.

In [9]:
!pip install symspellpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 9.0 MB/s eta 0:00:00


3.- Carga de recursos.


In [10]:
import pkg_resources
from symspellpy import SymSpell, Verbosity

# Descargamos algunos recursos para este práctica
!wget --no-check-certificate https://valencia.inf.um.es/valencia-plne/p1.zip
!unzip p1.zip

--2025-02-10 09:15:22--  https://valencia.inf.um.es/valencia-plne/p1.zip
Resolving valencia.inf.um.es (valencia.inf.um.es)... 155.54.204.133
Connecting to valencia.inf.um.es (valencia.inf.um.es)|155.54.204.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 694423 (678K) [application/zip]
Saving to: ‘p1.zip’

p1.zip              100%[===================>] 678.15K  1.16MB/s    in 0.6s    

2025-02-10 09:15:24 (1.16 MB/s) - ‘p1.zip’ saved [694423/694423]

Archive:  p1.zip
  inflating: CREA_80000.txt          
  inflating: Spanish.aff             
  inflating: Spanish.dic             
  inflating: ejerciciop1.txt         


4.- Carga de un diccionario en castellano para symspellpy.

symspellpy, como el resto de correctores, necesita diccionarios que utiliza para comprobar la corrección de las palabras. symspellpy utiliza diccionarios de frecuencia en los que cada palabra tiene asociado el número de veces que ha sido contabilizada en una serie de corpora.

Las sugerencias que nos va a proporcionar symspellpy van a depender en primer lugar de la distancia léxica entre el te´rmino analizado y los te´rminos que encuentre en el diccionario y en segundo lugar de la frecuancia con la aparecen dichos términos.

El diccionario que se proporciona está confeccionado a partir de un diccionario de frecuencia de la Real Academia de la Lengua e incluye los 80000 términos más frecuentes.


In [11]:
from google.colab import drive

# dictionary_file = data_dir_path + "/CREA_80000.txt"

dictionary_file =  "CREA_80000.txt"

# Traza ............................................................
f1 = open(dictionary_file, 'r', encoding ='utf-8')
for i in range(5):
  print(f1.readline(), end = '')
f1.close()

de,9999518
la,6277560
que,4681839
el,4569652
en,4234281


5.- Creación de un objeto symspell y carga de diccionario de frecuencias.

Se proporciona un ejemplo de creación de objeto con carga de frecuencias en inglés.

Symspellpy incorpora entre sus recursos un diccionario de frecuencias de palabras en inglés, que es el que se usa en el ejemplo.

Para trabajar en castellano tendrá que cargar el diccionario proporcionado en (4)

In [12]:
import pkg_resources
from symspellpy import SymSpell, Verbosity

# EJEMPLO. Diccionario de frecuencias en inglés................................

sym_spell_en = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
                        "symspellpy",
                        "frequency_dictionary_en_82_765.txt")

sym_spell_en.load_dictionary(dictionary_path, term_index=0, count_index=1)

# Traza.
from itertools import islice
print("sym_spell_en ", end = " ---> " )
print(list(islice(sym_spell_en.words.items(), 5)))
print("------------"*10)

# CASTELLANO ..................................................................

# En este caso tendrá que utilizar el diccionario obtenido en (4)
# sp_dictionary = "CREA_80000.txt"
sp_dictionary = data_dir_path + "/CREA_80000.txt"

sym_spell_sp = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)

sym_spell_sp.load_dictionary(sp_dictionary,
                             term_index=0,
                             count_index=1,
                             separator =',')

# Traza ......................................................................
from itertools import islice
print("sym_spell_sp ", end = " ---> " )
print(list(islice(sym_spell_sp.words.items(), 5)))
print("------------"*10)
print()

2025-02-10 09:15:27,415: E symspellpy.symspellpy] Dictionary file not found at /CREA_80000.txt.
ERROR:symspellpy.symspellpy:Dictionary file not found at /CREA_80000.txt.


sym_spell_en  ---> [('the', 23135851162), ('of', 13151942776), ('and', 12997637966), ('to', 12136980858), ('a', 9081174698)]
------------------------------------------------------------------------------------------------------------------------
sym_spell_sp  ---> []
------------------------------------------------------------------------------------------------------------------------



6.- Preprocesamiento previo a la detección y corrección de errores en palabras individuales.

Para facilitar la detección y corrección de errores en palabras individuales vamos a preprocesar los textos.

  * Eliminaremos los signos de puntuación con la función remove_punctuation.
  * Pasaremos todo a minúsculas.
  * Obtendremos las "palabras" del texto con la función tokenize.
  * Guardaremos en una nueva columna 'pals' las palabras correspondientes al texto con errores.

In [13]:
# Preprocesado, obtención de palabras

import string

sp_punctuation = string.punctuation + '¿' + '¡'

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in sp_punctuation])
    return punctuationfree

def tolower(text):
    return text.lower()

def tokenize(text):
    return text.split()

def preprocesado(text):
    return tokenize(tolower(remove_punctuation(text)))

datos['pals'] = datos['textos_con_errores'].apply(preprocesado)

#print(datos)
datos.head()


,textos_sin_errores,textos_con_errores,pals
0,La toma de posesión de Donald Trump como pres...,La tma de posesión de Donald Trump como presd...,"[la, tma, de, posesión, de, donald, trump, com..."
1,"Cerrar podrá mis ojos la postrera sombra, que...","Cerrar podrá mis hojos la postrera sombra, qu...","[cerrar, podrá, mis, hojos, la, postrera, somb..."


7.- Encontrar la ortografía más probable para una palabra dada. Función lookup.

Este ejercicio se da resuelto y se piden únicamente algunas modificaciones y reflexionar sobre los resultados.

Una vez que tenemos las palabras en una columna vamos a pasarles el corrector ortográfico y a imprimir las sugerencias que nos hace el corrector ortógrafico (se muestran sólo las que están a la mínima distancia de edición, Verbosity.CLOSEST).

En el primer texto tenemos los sigientes errores:
  * toma         -> tma
  * presidente   -> presdente
  * Capitolio    -> Capittolio
  * histórica    -> histórrica
  * tiene        -> teine
  * además       -> ademas
  * favor        -> fvaor
  * celebridades -> selevriddades
  * visión       -> vission

En el segundo texto tenemos los sigientes errores:
  * ojos           -> hojos
  * llevare        -> llevre
  * memoria        -> memorria
  * perder         -> preder
  * humor          -> rumor
  * gloriosamente  -> glorsamente
  * dejarán        -> dejaran
  * ceniza         -> cenisa
  * polvo          -> polbu

¿Qué falsos positivos y falsos negativos detecta?
Experimente con las palabras del texto con errores, introduzca nuevos errores, recargue los datos y observe los resultados que ofrece el corrector.

Cambie Verbosity.CLOSEST por Verbosity.TOP, elimine la condición *if suggestion.distance > 0* y vuelva a ejecutar el código.







In [14]:
def print_sugerencias (termino, distancia = 3):
  sugs = sym_spell_sp.lookup(termino,
                             Verbosity.CLOSEST, max_edit_distance=distancia)
  for suggestion in sugs:
    if suggestion.distance > 0:
       print('{:10}'.format(termino), end = " ---> ")
       print('{:10}'.format(suggestion.term),
             '{:3}'.format(suggestion.distance), suggestion.count)

for palabras in datos['pals']:
  for palabra in palabras:
    print_sugerencias(palabra)
  print("------------"*5)

# .....................................................................
print()
print("Con Verbosity. TOP ----> ")
print()
def print_sugerencias (termino, distancia = 3):
  sugs = sym_spell_sp.lookup(termino,
                             Verbosity.TOP, max_edit_distance=distancia)
  for suggestion in sugs:
       print('{:10}'.format(termino), end = " ---> ")
       print('{:10}'.format(suggestion.term),
             '{:3}'.format(suggestion.distance), suggestion.count)

for palabras in datos['pals']:
  for palabra in palabras:
    print_sugerencias(palabra)
  print("------------"*5)


------------------------------------------------------------
------------------------------------------------------------

Con Verbosity. TOP ----> 

------------------------------------------------------------
------------------------------------------------------------


8.- Vamos a añadir ahora tres columnas al frame de pandas:

 - Una con todas las palabras sugeridas por symspellpy (texto completo dividido en palabras): 'pals_symspellpy'. Para ello use la sugerencia más probable en lookup (Verbosity.TOP)
 - otra sólo con las palabras corregidas por symspellpy: 'corr_symspellpy'
 - y una más 'text_symspellpy' con el texto resultante de utilizar las palabras sugeridas por symspellpy (las de la columna 'pals_symspellpy).



In [16]:
def get_sugerencias(lista_terminos):
  l = []
  for termino in lista_terminos:
    suggestions = sym_spell_sp.lookup(termino, Verbosity.TOP, max_edit_distance=3)
    # Check if suggestions is not empty
    if suggestions:
      l.append(suggestions[0].term)
    else:
      # Handle the case where no suggestion is found (e.g., keep original term)
      l.append(termino)
  return l

def get_cambiadas (l_original, l_cambiada):
  l = [termino_cambiada
         for termino_original, termino_cambiada in zip(l_original, l_cambiada)
            if termino_original != termino_cambiada]
  return l

datos['pals_simspellpy'] = datos['pals'].apply(get_sugerencias)
datos['corr_simspellpy'] = datos.apply(lambda x:
                                          get_cambiadas(x['pals'],
                                                        x['pals_simspellpy']),
                                                        axis=1)
datos['text_simspellpy'] = datos['pals_simspellpy'].apply(lambda x: " ".join(x))

print()
print(datos['text_simspellpy'][0])
print()
print(datos['text_simspellpy'][1])
print("----------"*10)
datos.head()



la tma de posesión de donald trump como presdente de estados unidos se llevará a cabo el lunes veinte de enero en el capittolio de washington dc esta ceremonia hitórrica que se realiza cada cuatro años teine en su pista de invitados a figuras clave en la política mundial ademas de empresarios que busca ganarse el fvaor del nuevo comandante en jefe y selevridades que lo han apoyado y que comparten su vission para el futuro de la unión americana

cerrar podrá mis hojos la postrera sombra que me llevre el blanco día y podrá desatar esta alma mía hora a su afán ansioso lisonjera mas no de esa otra parte en la rivrera dejará la memorria en donde ardía nadar sabe mi llama la agua fría y preder el respeto a ley severa alma a quien todo un dios prisión ha sido venas que rumor a tanto fuego han dado médulas que han glorsamente ardido su cuerpo dejaran no su cuidado serán cenisa mas tendrán sentido polvo serán mas polbu enamorado
-----------------------------------------------------------------

,textos_sin_errores,textos_con_errores,pals,pals_simspellpy,corr_simspellpy,text_simspellpy
0,La toma de posesión de Donald Trump como pres...,La tma de posesión de Donald Trump como presd...,"[la, tma, de, posesión, de, donald, trump, com...","[la, tma, de, posesión, de, donald, trump, com...",[],la tma de posesión de donald trump como presde...
1,"Cerrar podrá mis ojos la postrera sombra, que...","Cerrar podrá mis hojos la postrera sombra, qu...","[cerrar, podrá, mis, hojos, la, postrera, somb...","[cerrar, podrá, mis, hojos, la, postrera, somb...",[],cerrar podrá mis hojos la postrera sombra que ...


9.- Adición de palabras al diccionario.

Si observamos los resultados del ejercicio anterior vemos que en el diccionario no se encuentran las palabras 'lisonjera' ni 'llevare', que son palabras correctas que no se hayan incluidas en el diccionario de frecuencias proporcionado.

Utilice la función create_dictionary_entry para añadirlas.

Una vez añadidas vuelva a ejecutar el código del punto 7. ¿Cómo utiliza lookup las palabras añadidas?


In [17]:
sym_spell_sp.create_dictionary_entry("lisonjera", 1)
sym_spell_sp.create_dictionary_entry("llevare", 1)


True

10.- Uso de la función lookup_compound para corrección de errores ortográficos en oraciones o en bloques de texto.

En los ejemplos y ejercicios vistos hasta ahora hemos considerado palabras individuales. Esta es una visión muy restringida de la corrección ortográfica ya que, por un lado, los errores implican a menudo a varias palabras y, por otro, hay errores que sólo pueden detectarse si disponemos de información sobre el contexto en el que se producen. En este ejercicio vamos a ver una parte de la funcionalidad de symspellpy ofrece para estos casos.

La función lookup_compound corrige tres tipos de errores en oraciones:

  - Espacio que insertado por error en una palabra correcta genera dos términos incorrectos.
  - Espacio omitido por error entre dos palabras correctas que genera un término incorrecto
  - Múltiples términos de entrada independientes con errores ortográficos.

Averigüe qué tipo de información contextual utiliza lookup_compound. Pruebe con las oraciones de abajo y con otras diferentes que estime oportunas y razone acerca del rendimiento de lookup compound y sobre los factores que pueden influir en el mismo.

*correjir ortografia enespañol correc tamente*

*La toma deposesión de Donald Trump como presdente de Estados Unidos se llevará a cabo el lunes, dos de enero en el Capi tolio*
    
*Alma a quien todo un Dios prisión ha sido, benas que rumor a tanto fuego han dado, médula que han gloriosa mente ardido, su cuerpo dejarán, nosu cuidado*


In [19]:
input_terms = [
    "correjir ortografia enespañol correc tamente ",
    "La toma deposesión de Donald Trump como presdente de Estados Unidos" \
    "se llevará a cabo el lunes, dos de enero en el Capi tolio",
    "La sombra que me llevre el blanco día, alma dequien todo un Dios prisión" \
    "hasido, venas quehumor a tanto fuego, médulasque han gloriosa mente ardido," \
    "su cuerpo dejarán, no sucuidado",
    "La asginatura de prosesamiento de lenguajenatural escrito haze trabajar" \
    " y aprehnder perfectam ente procesamiengo de teexto"
    ]

for input_term in input_terms:
  print(input_term)
  print("Suggestions ---> ")
  suggestions = sym_spell_sp.lookup_compound(input_term, max_edit_distance=2)
  for suggestion in suggestions:
    print(suggestion.term)
  print("---"*16)


correjir ortografia enespañol correc tamente 
Suggestions ---> 
correjir ortografia enespañol correc tamente
------------------------------------------------
La toma deposesión de Donald Trump como presdente de Estados Unidosse llevará a cabo el lunes, dos de enero en el Capi tolio
Suggestions ---> 
la toma deposesión de donald trump como presdente de estados unidosse llevare cabo el lunes dos de enero en el capi tolio
------------------------------------------------
La sombra que me llevre el blanco día, alma dequien todo un Dios prisiónhasido, venas quehumor a tanto fuego, médulasque han gloriosa mente ardido,su cuerpo dejarán, no sucuidado
Suggestions ---> 
la sombra que me llevare el blanco día alma dequien todo un dios prisiónhasido venas quehumor a tanto fuego médulasque han gloriosa mente ardido su cuerpo dejarán no sucuidado
------------------------------------------------
La asginatura de prosesamiento de lenguajenatural escrito haze trabajar y aprehnder perfectam ente procesa

11.- Segmentación de textos

Symspellpy es capaz de segmentar textos en los que no hay separación entre las palabras mediante la función word_segmentation.

Averigüe qué tipo de información contextual utiliza word_segementation. Pruebe con las oraciones de abajo y con otras diferentes que estime oportunas y razone acerca del rendimiento de word_segmentation y sobre los factores que pueden influir en el mismo.

*divideestaoraciónenpalabrasindividuales*

*correjirortografiaenespañolcorrectamente*

*corregirortografíaenespañolcorrectamente*

*LatomadeposesióndeDonaldTrumpcomopresdentedeEstadosUnidos*

*AlmaaquientodounDiosprisiónhasidovenasquehumorafuegohandado*


In [20]:
input_terms = ["divideestaoraciónenpalabrasindividuales",
               "correjirortografiaenespañolcorrectamente",
               "corregirortografíaenespañolcorrectamente",
               "LatomadeposesióndeDonaldTrumpcomopresdentedeEstadosUnidos",
               "AlmaaquientodounDiosprisiónhasidovenasquehumorafuegohandado"
              ]
for input_term in input_terms:
  print(input_term)
  result= sym_spell_sp.word_segmentation(input_term)
  print(f"Segmented Sentence: {result.corrected_string}")
  print(f"Sum of Edit Distances: {result.distance_sum}")
  print(f"Sum of Log Probabilities: {result.log_prob_sum}")
  print("---"*16)

divideestaoraciónenpalabrasindividuales
Segmented Sentence: div ideestaor aciónenpa labrasind ividuales
Sum of Edit Distances: 43
Sum of Log Probabilities: -94.0534249815
------------------------------------------------
correjirortografiaenespañolcorrectamente
Segmented Sentence: corr ejirortog rafiaenes pañolcorr ectamente
Sum of Edit Distances: 44
Sum of Log Probabilities: -95.0534249815
------------------------------------------------
corregirortografíaenespañolcorrectamente
Segmented Sentence: corr egirortog rafíaenes pañolcorr ectamente
Sum of Edit Distances: 44
Sum of Log Probabilities: -95.0534249815
------------------------------------------------
LatomadeposesióndeDonaldTrumpcomopresdentedeEstadosUnidos
Segmented Sentence: Lat omadepose sióndeDon aldTrumpc omopresde ntedeEsta dosUnidos
Sum of Edit Distances: 63
Sum of Log Probabilities: -134.0747949741
------------------------------------------------
AlmaaquientodounDiosprisiónhasidovenasquehumorafuegohandado
Segmented Sentenc

12.- Generación y uso de un diccionario a partir de un corpus.

En el siguiente ejemplo vamos a generar un diccionario a partir de un corpus de recetas de cocina.


In [21]:
# Descargamos un corpus de recetas de cocina
!wget --no-check-certificate https://valencia.inf.um.es/valencia-plne/corpusRecetasv3.tgz
!tar -xzf corpusRecetasv3.tgz

--2025-02-10 09:19:12--  https://valencia.inf.um.es/valencia-plne/corpusRecetasv3.tgz
Resolving valencia.inf.um.es (valencia.inf.um.es)... 155.54.204.133
Connecting to valencia.inf.um.es (valencia.inf.um.es)|155.54.204.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17305355 (17M) [application/x-gzip]
Saving to: ‘corpusRecetasv3.tgz’

corpusRecetasv3.tgz 100%[===================>]  16.50M  11.8MB/s    in 1.4s    

2025-02-10 09:19:14 (11.8 MB/s) - ‘corpusRecetasv3.tgz’ saved [17305355/17305355]



In [22]:
import os
from symspellpy.symspellpy import SymSpell, Verbosity

corpus_folder = "./corpusRecetasv3"
dictionary_path = "./symspell_corpusRecetas_dictionary.txt"

# Creamos un diccionario a partir del corpus descargado
symspell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
# Leer todos los archivos de texto en la carpeta
for file_name in os.listdir(corpus_folder):
  if file_name.endswith(".txt"):
    file_path = os.path.join(corpus_folder, file_name)
    symspell.create_dictionary(file_path)

# Guardar el diccionario generado manualmente
with open(dictionary_path, "w", encoding="utf-8") as f:
  for key, count in symspell.words.items():
    f.write(f"{key} {count}\n")
print(f"Diccionario guardado en {dictionary_path}")


Diccionario guardado en ./symspell_corpusRecetas_dictionary.txt


In [23]:
max_edit_distance = 3
#Creamos un diccionario y lo cargamos
symspell_corpus = SymSpell(max_dictionary_edit_distance= max_edit_distance, prefix_length=7)
symspell_corpus.load_dictionary(dictionary_path, term_index=0, count_index=1)

# Pruebas
test_words = ["cane", "pallo", "espageti"]
for word in test_words:
    suggestions = symspell_corpus.lookup(word, Verbosity.CLOSEST, max_edit_distance)
    print(f"Pruebas con la palabra: '{word}'")
    if suggestions:
        for suggestion in suggestions:
            print(f"Corrección sugerida: {suggestion.term}, Distancia: {suggestion.distance}, Frecuencia: {suggestion.count}")
    else:
        print("No se encontraron correcciones sugeridas.")


Pruebas con la palabra: 'cane'
Corrección sugerida: carne, Distancia: 1, Frecuencia: 15867
Corrección sugerida: cake, Distancia: 1, Frecuencia: 714
Corrección sugerida: cabe, Distancia: 1, Frecuencia: 257
Corrección sugerida: cae, Distancia: 1, Frecuencia: 56
Corrección sugerida: cine, Distancia: 1, Frecuencia: 36
Corrección sugerida: caen, Distancia: 1, Frecuencia: 12
Corrección sugerida: gane, Distancia: 1, Frecuencia: 8
Corrección sugerida: pane, Distancia: 1, Frecuencia: 5
Corrección sugerida: cafe, Distancia: 1, Frecuencia: 5
Corrección sugerida: cone, Distancia: 1, Frecuencia: 2
Corrección sugerida: can, Distancia: 1, Frecuencia: 2
Corrección sugerida: canse, Distancia: 1, Frecuencia: 1
Corrección sugerida: lane, Distancia: 1, Frecuencia: 1
Corrección sugerida: crne, Distancia: 1, Frecuencia: 1
Corrección sugerida: case, Distancia: 1, Frecuencia: 1
Corrección sugerida: cale, Distancia: 1, Frecuencia: 1
Corrección sugerida: care, Distancia: 1, Frecuencia: 1
Corrección sugerida: ca

12.- Corrección de errores con Hunspell.

symspellpy es sólo una de las librerías de corrección ortográfica que tenemos a nuestra disposición. En los siguientes ejercicios vamos a trabajar con la librería hunspell (https://hunspell.github.io/).

En este ejercicio vamos a:
  - Instalar humspell y cargar diccionarios en español.
  - Trabajar sobre la columna 'pals'para ver que correcciones nos sugiere en los textos proporcionados (como ya se ha hecho con symspellpy).



In [26]:
!sudo apt-get install libhunspell-dev
!pip install hunspell

# paths a los diccionarios de hunspell
dic_path = "./Spanish.dic"
aff_path = "./Spanish.aff"

# dic_path = data_dir_path + "/Spanish.dic"
# aff_path = data_dir_path + "/Spanish.aff"


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libhunspell-dev is already the newest version (1.7.0-4build1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


13.- Creación de un objeto HunSpell para obtener las correcciones sugeridas.

  - Cremoas objeto HunSpell suministrándole los dicccionarios en español.
  - Aplicamos el método suggest sobre la columna 'pals' para ver que correcciones nos sugiere en los textos proporcionados.

In [28]:
import hunspell
dic = hunspell.HunSpell(dic_path, aff_path)

for palabras in datos['pals']:
  for palabra in palabras:
    candidatos = dic.suggest(palabra)
    if palabra != candidatos[0]:
       print(palabra, end = ": correccion sugerida --->   ")
       print(candidatos[0])
       print(palabra, end = ": posibilidades       ---> ")
       print(candidatos)
       print("------------"*5)


tma: correccion sugerida --->   ta
tma: posibilidades       ---> ['ta', 'tema', 'toma', 'tima', 'ama', 'tea', 'toa', 'tía', 'ritma']
------------------------------------------------------------
donald: correccion sugerida --->   donad
donald: posibilidades       ---> ['donad', 'donala', 'donalo', 'McDonald', 'baldona', 'hondonal', 'algodonal']
------------------------------------------------------------
trump: correccion sugerida --->   trumao
trump: posibilidades       ---> ['trumao']
------------------------------------------------------------
presdente: correccion sugerida --->   presente
presdente: posibilidades       ---> ['presente', 'presidente', 'presiente', 'precedente', 'presidenta', 'presciente', 'presentate']
------------------------------------------------------------
llevará: correccion sugerida --->   llevara
llevará: posibilidades       ---> ['llevara', 'llevará', 'llevar', 'levará', 'llevarán', 'llevarás', 'llevare', 'elevará', 'llenará', 'llegará', 'llevaré', 'lle var

14.- Adición de columnas a datos con los resultados de aplicar HunSpell a las palabras de los textos.

Vamos a añadir tres columnas al frame de pandas análogas a las que ya creamos con simspellpy (ejercicio 8):

- Una columna con todas las palabras sugeridas por pyspellchecker (texto completo dividido en palabras): 'pals_hunspell'. Para ello, como en el ejercicio anterior, usaremos el método suggest(word).
- Otra columna sólo con las palabras corregidas por HunSpell: 'corr_hunspell'.
- Y una columna más, 'text_hunspell', con el texto resultante de utilizar las palabras sugeridas por HunSpel (las que hemos puesto en 'pals_hunspell).

Podrá observar que los resultados son diferentes según el corrector utilizado. Esto puede deberse a muchos factores, aunque en nuestro caso hay uno muy evidente: se han utilizado diccionarios diferentes.

In [29]:
def get_sugerencias_hunsspell (lista_terminos):
  l = [dic.suggest(termino)[0]
               if dic.suggest(termino)[0] is not None
               else termino for termino in lista_terminos]
  return l


def get_cambiadas_hunsspell (l_original, l_cambiada):
  l = [termino_cambiada
         for termino_original, termino_cambiada in zip(l_original, l_cambiada)
            if termino_original != termino_cambiada]
  return l

datos['pals_hunsspell'] = datos['pals'].apply(get_sugerencias_hunsspell)
datos['corr_hunsspell'] = datos.apply(lambda x:
                                          get_cambiadas_hunsspell(x['pals'],
                                                        x['pals_hunsspell']),
                                                        axis=1)

datos['text_hunsspell'] = datos['pals_hunsspell'].apply(lambda x: " ".join(x))

print()
print(datos['text_hunsspell'][0])
print()
print(datos['text_hunsspell'][1])
print("----------"*10)

datos.head()


la ta de posesión de donad trumao como presente de estados unidos se llevara ea cabo el lunes veinte de enero en el capitolio de Washington d esta ceremonia histórica que se realiza cada cuatro anos tiene en su pista de invitados ea figuras clave en la política mundial ademas de empresarios que busca ganarse el favor del nuevo comandante en jefe u severidades que lo han apoyado u que comparten su visiona para el futuro de la unión americana

cerrar podrá mis hijos la postrera sombra que me lleve el blanco día u podrá desatar esta alma miá hora ea su afán ansioso lisonjera mas no de esa otra parte en la rivera dejara la memoria en donde ardía nadar sabe mi llama la agua fría u perder el respeto ea ley severa alma ea quien todo un dios prisión ha sido venas que rumor ea tanto fuego han dado médulas que han gloriosamente ardido su cuerpo dejaran no su cuidado serán cenias mas tendrán sentido polvo serán mas pol bu enamorado
----------------------------------------------------------------

,textos_sin_errores,textos_con_errores,pals,pals_simspellpy,corr_simspellpy,text_simspellpy,pals_hunsspell,corr_hunsspell,text_hunsspell
0,La toma de posesión de Donald Trump como pres...,La tma de posesión de Donald Trump como presd...,"[la, tma, de, posesión, de, donald, trump, com...","[la, tma, de, posesión, de, donald, trump, com...",[],la tma de posesión de donald trump como presde...,"[la, ta, de, posesión, de, donad, trumao, como...","[ta, donad, trumao, presente, llevara, ea, cap...",la ta de posesión de donad trumao como present...
1,"Cerrar podrá mis ojos la postrera sombra, que...","Cerrar podrá mis hojos la postrera sombra, qu...","[cerrar, podrá, mis, hojos, la, postrera, somb...","[cerrar, podrá, mis, hojos, la, postrera, somb...",[],cerrar podrá mis hojos la postrera sombra que ...,"[cerrar, podrá, mis, hijos, la, postrera, somb...","[hijos, lleve, u, miá, ea, rivera, dejara, mem...",cerrar podrá mis hijos la postrera sombra que ...


15.- Finalmente, vamos a visualizar cómo quedan los textos con las correciones hechas con ambos correctores.

In [30]:
for row in datos.itertuples():
  print("Texto sin errores ----> ", end = " ")
  print(datos['textos_sin_errores'][row.Index])
  print("Texto con errores ----> ", end = " ")
  print(datos['textos_con_errores'][row.Index])
  print("Texto symspellpy  ----> ", end = " ")
  print(datos['text_simspellpy'][row.Index])
  print("Texto hunspell    ----> ", end = " ")
  print(datos['text_hunsspell'][row.Index])
  print()
  print("------------"*5)

Texto sin errores ---->   La toma de posesión de Donald Trump como presidente de Estados Unidos se llevará a cabo el lunes veinte de enero en el Capitolio de Washington D.C. Esta ceremonia histórica, que se realiza cada cuatro años, tiene en su lista de invitados a figuras clave en la política mundial, además de empresarios que buscan ganarse el favor del nuevo comandante en jefe, y celebridades que lo han apoyado y que comparten su visión para el futuro de la unión americana.
Texto con errores ---->   La tma de posesión de Donald Trump como presdente de Estados Unidos se llevará a cabo el lunes, veinte de enero en el Capittolio de Washington D.C. Esta ceremonia hitórrica, que se realiza cada cuatro años, teine en su pista de invitados a figuras clave en la política mundial, ademas de empresarios que busca ganarse el fvaor del nuevo comandante en jefe, y selevridades que lo han apoyado y que comparten su vission para el futuro de la unión americana.
Texto symspellpy  ---->  la tma de p